# Regression

## initialize and get dataset

In [95]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


## input and output layer

In [96]:
# place holder is like blob in Caffe
# x is input layer and y_ as output layer (None means it depends on the size of batch)
# W is the single middle layer
x = tf.placeholder(tf.float32, [None, 784]) # shape is optional but it avoid errors
y_ = tf.placeholder(tf.float32, [None, 10])

## middle layers

In [97]:
# variables
W = tf.Variable(tf.zeros([784,10]))   # node weights
b = tf.Variable(tf.zeros([10]))       # biases 
# the irony here is this that the value of these variables is not assigned
# it is just like pointer, it will be assigned on call to "initialize_all_variables" later

## output layer

In [98]:
# heh easy :D
# it is just a simple regression model, it multiply matrices input and weights and add bias
# then take a softmax out of it
y = tf.nn.softmax(tf.matmul(x, W) + b)

## cost function

In [99]:
# reduce here is like Map-Reduce
# calculate mean of errors
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))

## optimization algorithm

In [100]:
# Gradiant Decent Optimizer
# with step size 0.5
# that tries to minimize using cross_entropy tensor (it is function here)
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

## initializer

In [101]:
# it is an operation
# but still it did not run yet
init = tf.initialize_all_variables()

## session 

In [102]:
# every thing is done under session
# a session is a connection to outer code (in C++ or Cuda)
sess = tf.InteractiveSession()
#sess = tf.Session()

In [103]:
# now it runs initialization
sess.run(init)

## training

In [104]:
# run training steps
for i in range(1000):
  batch_xs, batch_ys = mnist.train.next_batch(100)
  sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})

## evaluation

In [105]:
# argmax gives the index of highest value in array
# we use it find the best choice
# y is our prediction and y_ is the real values
# the result is a list of booleans showing if the results are correct or not
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))

In [106]:
# convert booleans to float and get mean
# it is still an operation not a call for a function
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [107]:
print(sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

0.9172


# Convolutional Network

In [108]:
# for initialization of variables
def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

In [109]:
# functions for doing convolutions
# stride is 1 and we have 0 padding
# then a pool
def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

In [110]:
# 5*5 filter     1 channel     32 output (means 32 filters)
W_conv1 = weight_variable([5, 5, 1, 32]) # weights
b_conv1 = bias_variable([32])            # bias

In [111]:
# make image of 28*28 with 1 color chanel
x_image = tf.reshape(x, [-1,28,28,1])

## First Convolutional Layer

In [112]:
# conv layer with relu
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
# pooling layer
h_pool1 = max_pool_2x2(h_conv1)

## Second Convolutional Layer

In [113]:
# second convolutional layer
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

## Full Connected layer

In [114]:
W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)


## Drop Out

In [115]:
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

## Readout Layer 

In [116]:
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y_conv=tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

## Evaluate

In [117]:
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y_conv), reduction_indices=[1]))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [118]:
sess.run(tf.initialize_all_variables())


In [119]:
for i in range(20000):
  batch = mnist.train.next_batch(50)
  if i%100 == 0:
    train_accuracy = accuracy.eval(feed_dict={
        x:batch[0], y_: batch[1], keep_prob: 1.0})
    print("step %d, training accuracy %g"%(i, train_accuracy))
  train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})



step 0, training accuracy 0.1
step 100, training accuracy 0.88
step 200, training accuracy 0.94
step 300, training accuracy 0.9
step 400, training accuracy 0.94
step 500, training accuracy 0.86
step 600, training accuracy 0.96
step 700, training accuracy 0.96
step 800, training accuracy 0.98
step 900, training accuracy 0.92
step 1000, training accuracy 0.98
step 1100, training accuracy 0.94
step 1200, training accuracy 0.96
step 1300, training accuracy 0.92
step 1400, training accuracy 0.96
step 1500, training accuracy 0.94
step 1600, training accuracy 1
step 1700, training accuracy 0.98
step 1800, training accuracy 1
step 1900, training accuracy 0.96
step 2000, training accuracy 0.98
step 2100, training accuracy 0.94
step 2200, training accuracy 1
step 2300, training accuracy 0.98


KeyboardInterrupt: 

In [120]:
print("test accuracy %g"%accuracy.eval(feed_dict={
    x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))


test accuracy 0.9793
